# Web App Data Prep

This notebook prepares the cleaned flight dataset for the web app database. It builds dimension tables (airlines, airports, aircraft types) and fact tables (routes and route metrics), then writes Postgres DDL and seed INSERT statements into `./sql_statements`.

Data source: `./clean_data/final_flight_data.csv`


## Environment setup
Install required Python packages for data wrangling and file generation.

In [1]:
%pip install pandas numpy


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Load dataset
Read the cleaned flight dataset once so all export steps share the same dataframe.

In [2]:
import os
import pandas as pd
import numpy as np
import hashlib
import uuid
import re
import time
import requests
import math
import random
from datetime import date

df = pd.read_csv('./clean_data/final_flight_data.csv')

## Airlines table and seed data
Filter and normalize airline fields, then emit Postgres DDL and INSERT statements for the `airlines` table.

In [3]:
# Cell 1 — filter/normalize airlines from df + generate Postgres SQL files into ./sql_statements


# df = ...  # your raw dataframe

SQL_DIR = "./sql_statements"
os.makedirs(SQL_DIR, exist_ok=True)

# 1) Extract & normalize airline dimension
airlines_df = (
    df[[
        "airline_id",
        "airline_name",
        "airline_iata",
        "airline_icao",
        "airline_call_sign",
        "airline_country",
        "airline_active",
    ]]
    .drop_duplicates()
    .copy()
)

# cleanup
for c in ["airline_name", "airline_iata", "airline_icao", "airline_call_sign", "airline_country"]:
    airlines_df[c] = airlines_df[c].astype("string").str.strip().replace({"": pd.NA})

airlines_df["airline_active"] = airlines_df["airline_active"].fillna(False).astype(bool)

# recommended filters: active only + require at least one code
airlines_df = airlines_df[
    (airlines_df["airline_active"])
    & (airlines_df["airline_iata"].notna() | airlines_df["airline_icao"].notna())
].copy()

# uppercase codes
for c in ["airline_iata", "airline_icao"]:
    airlines_df[c] = airlines_df[c].str.upper()

# 2) Generate SQL files (DDL + inserts)
ddl_sql = """\
-- airlines table (Postgres)
CREATE TABLE IF NOT EXISTS airlines (
  airline_id   TEXT PRIMARY KEY,
  name         TEXT,
  iata_code    TEXT,
  icao_code    TEXT,
  call_sign    TEXT,
  country      TEXT,
  is_active    BOOLEAN NOT NULL DEFAULT TRUE,
  logo_path    TEXT
);

CREATE UNIQUE INDEX IF NOT EXISTS airlines_iata_code_uq
  ON airlines (iata_code) WHERE iata_code IS NOT NULL;

CREATE UNIQUE INDEX IF NOT EXISTS airlines_icao_code_uq
  ON airlines (icao_code) WHERE icao_code IS NOT NULL;
"""

def sql_quote(v):
    if v is None or (isinstance(v, float) and pd.isna(v)) or (isinstance(v, pd._libs.missing.NAType)):
        return "NULL"
    s = str(v).replace("'", "''")
    return f"'{s}'"

insert_rows = []
for r in airlines_df.itertuples(index=False):
    insert_rows.append(
        "INSERT INTO airlines (airline_id, name, iata_code, icao_code, call_sign, country, is_active)\n"
        f"VALUES ({sql_quote(r.airline_id)}, {sql_quote(r.airline_name)}, {sql_quote(r.airline_iata)}, "
        f"{sql_quote(r.airline_icao)}, {sql_quote(r.airline_call_sign)}, {sql_quote(r.airline_country)}, "
        f"{'TRUE' if bool(r.airline_active) else 'FALSE'});\n"
    )

inserts_sql = "-- airlines seed data\n" + "".join(insert_rows)

with open(os.path.join(SQL_DIR, "001_create_airlines.sql"), "w", encoding="utf-8") as f:
    f.write(ddl_sql)

with open(os.path.join(SQL_DIR, "002_insert_airlines.sql"), "w", encoding="utf-8") as f:
    f.write(inserts_sql)

print(f"Wrote SQL files to: {os.path.abspath(SQL_DIR)}")
print(f"Airlines rows exported: {len(airlines_df):,}")


Wrote SQL files to: /Users/darius/Documents/projects/portfolio/flighter/data-mining/sql_statements
Airlines rows exported: 174


## Optional: download airline logos
Fetch airline logos into `./airline_images` for UI use. This cell is intentionally skipped by default.

In [4]:
%%script echo skipping airline logo downloads, run manually as needed
# Cell 2 — download airline logos to ./airline_images (no DB writes), only run manually as needed



IMG_DIR = "./airline_images"
os.makedirs(IMG_DIR, exist_ok=True)

# Use the filtered airlines_df from Cell 1
# We'll primarily use IATA; fall back to ICAO only for filename (many logo sources are IATA-based)
def safe_code(x: str) -> str | None:
    if x is None or pd.isna(x):
        return None
    x = str(x).strip().upper()
    return x if re.fullmatch(r"[A-Z0-9]{2,4}", x) else None

# Aviasales/Travelpayouts logo URL pattern (IATA-based, PNG)
def aviasales_logo_url(iata: str, size: int = 200) -> str:
    return f"http://img.wway.io/pics/root/{iata}@png?exar=1&rs=fit:{size}:{size}"

session = requests.Session()
session.headers.update({"User-Agent": "flight-booking-app/0.1 (logo fetcher)"})

downloaded = 0
skipped = 0
failed = 0

for r in airlines_df.itertuples(index=False):
    iata = safe_code(r.airline_iata)
    icao = safe_code(r.airline_icao)

    # this source is IATA-based; skip if no IATA
    if not iata:
        skipped += 1
        continue

    out_path = os.path.join(IMG_DIR, f"{iata}.png")
    if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
        skipped += 1
        continue

    try:
        url = aviasales_logo_url(iata, size=200)
        resp = session.get(url, timeout=20)

        if resp.status_code != 200 or not resp.content:
            failed += 1
            continue

        ctype = (resp.headers.get("Content-Type") or "").lower()
        if "image" not in ctype:
            failed += 1
            continue

        with open(out_path, "wb") as f:
            f.write(resp.content)

        downloaded += 1
        time.sleep(0.05)  # be polite

    except Exception:
        failed += 1

print(f"Saved logos to: {os.path.abspath(IMG_DIR)}")
print(f"Downloaded: {downloaded:,} | Skipped: {skipped:,} | Failed: {failed:,}")


skipping airline logo downloads, run manually as needed


## Airports table and seed data
Create a unified airports dimension from both source and destination columns, then write DDL and INSERTs.

In [5]:
# Cell 1 — generate Postgres SQL for airports into ./sql_statements

SQL_DIR = "./sql_statements"
os.makedirs(SQL_DIR, exist_ok=True)

def sql_quote(v):
    if v is None or (isinstance(v, float) and pd.isna(v)) or (pd.isna(v) if not isinstance(v, str) else False):
        return "NULL"
    s = str(v).replace("'", "''")
    return f"'{s}'"

def to_bool(v):
    if pd.isna(v):
        return "NULL"
    return "TRUE" if bool(v) else "FALSE"

def to_num(v):
    if pd.isna(v) or v is None:
        return "NULL"
    return str(v)

# ---- Build airports dimension from BOTH source + destination columns
src_airports = pd.DataFrame({
    "airport_id": df["route_source_airport_id"],
    "name": df["source_port_name"],
    "city": df["source_port_city"],
    "country": df["source_port_country"],
    "iata_code": df["source_port_iata"],
    "icao_code": df["source_port_icao"],
    "latitude": df["source_port_latitude"],
    "longitude": df["source_port_longitude"],
    "timezone": df["source_port_timezone"],
    "database_timezone": df["source_port_database_timezone"],
    "type": df["source_Type"],
})

dst_airports = pd.DataFrame({
    "airport_id": df["route_destination_airport_id"],
    "name": df["destination_port_name"],
    "city": df["destination_port_city"],
    "country": df["destination_port_country"],
    "iata_code": df["destination_port_iata"],
    "icao_code": df["destination_port_icao"],
    "latitude": df["destination_port_latitude"],
    "longitude": df["destination_port_longitude"],
    "timezone": df["destination_port_timezone"],
    "database_timezone": df["destination_port_database_timezone"],
    "type": df["destination_Type"],
})

airports_df = (
    pd.concat([src_airports, dst_airports], ignore_index=True)
    .drop_duplicates(subset=["airport_id"])
    .copy()
)

# cleanup
for c in ["name","city","country","iata_code","icao_code","timezone","database_timezone","type"]:
    airports_df[c] = airports_df[c].astype("string").str.strip().replace({"": pd.NA})
for c in ["iata_code","icao_code"]:
    airports_df[c] = airports_df[c].str.upper()

# ---- DDL
airports_ddl = """\
-- airports table (Postgres)
CREATE TABLE IF NOT EXISTS airports (
  airport_id        BIGINT PRIMARY KEY,
  name              TEXT,
  city              TEXT,
  country           TEXT,
  iata_code          TEXT,
  icao_code          TEXT,
  latitude          DOUBLE PRECISION,
  longitude         DOUBLE PRECISION,
  timezone          TEXT,
  database_timezone TEXT,
  type              TEXT
);

CREATE UNIQUE INDEX IF NOT EXISTS airports_iata_code_uq
  ON airports (iata_code) WHERE iata_code IS NOT NULL;

CREATE UNIQUE INDEX IF NOT EXISTS airports_icao_code_uq
  ON airports (icao_code) WHERE icao_code IS NOT NULL;

CREATE INDEX IF NOT EXISTS airports_country_city_idx
  ON airports (country, city);
"""

# ---- INSERTs
rows = []
for r in airports_df.itertuples(index=False):
    rows.append(
        "INSERT INTO airports (airport_id, name, city, country, iata_code, icao_code, latitude, longitude, timezone, database_timezone, type)\n"
        f"VALUES ({to_num(r.airport_id)}, {sql_quote(r.name)}, {sql_quote(r.city)}, {sql_quote(r.country)}, "
        f"{sql_quote(r.iata_code)}, {sql_quote(r.icao_code)}, {to_num(r.latitude)}, {to_num(r.longitude)}, "
        f"{sql_quote(r.timezone)}, {sql_quote(r.database_timezone)}, {sql_quote(r.type)});\n"
    )

airports_inserts = "-- airports seed data\n" + "".join(rows)

with open(os.path.join(SQL_DIR, "003_create_airports.sql"), "w", encoding="utf-8") as f:
    f.write(airports_ddl)

with open(os.path.join(SQL_DIR, "004_insert_airports.sql"), "w", encoding="utf-8") as f:
    f.write(airports_inserts)

print(f"Wrote airports SQL to: {os.path.abspath(SQL_DIR)} | rows: {len(airports_df):,}")


Wrote airports SQL to: /Users/darius/Documents/projects/portfolio/flighter/data-mining/sql_statements | rows: 1,372


## Aircraft types, routes, and route metrics
Generate aircraft type rows, deterministic route IDs, and route metrics, then write all related SQL files.

In [6]:
# Cell 2 — generate Postgres SQL for aircraft_types (planes), routes, and route_metrics into ./sql_statements

SQL_DIR = "./sql_statements"
os.makedirs(SQL_DIR, exist_ok=True)

def sql_quote(v):
    if v is None or (isinstance(v, float) and pd.isna(v)) or (pd.isna(v) if not isinstance(v, str) else False):
        return "NULL"
    s = str(v).replace("'", "''")
    return f"'{s}'"

def to_num(v):
    if v is None or (isinstance(v, float) and pd.isna(v)) or (pd.isna(v) if not isinstance(v, str) else False):
        return "NULL"
    return str(v)

def to_bool(v):
    if v is None or (isinstance(v, float) and pd.isna(v)) or (pd.isna(v) if not isinstance(v, str) else False):
        return "NULL"
    return "TRUE" if bool(v) else "FALSE"

def stable_uuid_from_key(key: str) -> str:
    """Deterministic UUID from a string key."""
    return str(uuid.uuid5(uuid.NAMESPACE_URL, key))

def stable_route_id(airline_id, src_id, dst_id, stops, plane_iso):
    # deterministic id (UUID) from a hash of natural keys
    key = f"ROUTE|{airline_id}|{src_id}|{dst_id}|{stops}|{plane_iso}"
    return stable_uuid_from_key(key)

# -------------------------
# aircraft_types (planes)
# -------------------------
planes_df = (
    df[[
        "plane_iso",
        "plane_name",
        "aircraft_name",
        "manufacturer",
        "category",
        "fuel_litre_per_100km_per_passenger",
        "capacity_min",
        "capacity_max",
        "range_nm",
        "co2_g_per_pax_mile",
    ]]
    .drop_duplicates(subset=["plane_iso"])
    .copy()
)

for c in ["plane_iso","plane_name","aircraft_name","manufacturer","category"]:
    planes_df[c] = planes_df[c].astype("string").str.strip().replace({"": pd.NA})
planes_df["plane_iso"] = planes_df["plane_iso"].str.upper()

planes_ddl = """\
-- aircraft_types table (Postgres)
CREATE TABLE IF NOT EXISTS aircraft_types (
  plane_iso                         TEXT PRIMARY KEY,
  plane_name                        TEXT,
  aircraft_name                     TEXT,
  manufacturer                      TEXT,
  category                          TEXT,
  fuel_litre_per_100km_per_passenger DOUBLE PRECISION,
  capacity_min                      INTEGER,
  capacity_max                      INTEGER,
  range_nm                          INTEGER,
  co2_g_per_pax_mile                DOUBLE PRECISION
);

CREATE INDEX IF NOT EXISTS aircraft_types_mfr_cat_idx
  ON aircraft_types (manufacturer, category);
"""

planes_rows = []
for r in planes_df.itertuples(index=False):
    planes_rows.append(
        "INSERT INTO aircraft_types (plane_iso, plane_name, aircraft_name, manufacturer, category, "
        "fuel_litre_per_100km_per_passenger, capacity_min, capacity_max, range_nm, co2_g_per_pax_mile)\n"
        f"VALUES ({sql_quote(r.plane_iso)}, {sql_quote(r.plane_name)}, {sql_quote(r.aircraft_name)}, "
        f"{sql_quote(r.manufacturer)}, {sql_quote(r.category)}, {to_num(r.fuel_litre_per_100km_per_passenger)}, "
        f"{to_num(r.capacity_min)}, {to_num(r.capacity_max)}, {to_num(r.range_nm)}, {to_num(r.co2_g_per_pax_mile)});\n"
    )

planes_inserts = "-- aircraft_types seed data\n" + "".join(planes_rows)

with open(os.path.join(SQL_DIR, "005_create_aircraft_types.sql"), "w", encoding="utf-8") as f:
    f.write(planes_ddl)

with open(os.path.join(SQL_DIR, "006_insert_aircraft_types.sql"), "w", encoding="utf-8") as f:
    f.write(planes_inserts)

print(f"Wrote aircraft_types SQL | rows: {len(planes_df):,}")

# -------------------------
# routes + route_metrics
# -------------------------
routes_cols = [
    "airline_id",
    "route_source_airport_id",
    "route_destination_airport_id",
    "route_stops",
    "route_plane_iso",
    "distance_km",
    "distance_miles",
    "is_international",
    "co2_total_kg",
]

routes_raw = df[routes_cols].drop_duplicates().copy()

# normalize codes
routes_raw["route_plane_iso"] = routes_raw["route_plane_iso"].astype("string").str.strip().str.upper()
routes_raw["airline_id"] = routes_raw["airline_id"].astype("string").str.strip()

# deterministic route_id
routes_raw["route_id"] = routes_raw.apply(
    lambda x: stable_route_id(
        x["airline_id"],
        x["route_source_airport_id"],
        x["route_destination_airport_id"],
        x["route_stops"],
        x["route_plane_iso"],
    ),
    axis=1,
)

routes_df = routes_raw[[
    "route_id",
    "airline_id",
    "route_source_airport_id",
    "route_destination_airport_id",
    "route_stops",
    "route_plane_iso",
]].drop_duplicates(subset=["route_id"]).copy()

route_metrics_df = routes_raw[[
    "route_id",
    "distance_km",
    "distance_miles",
    "is_international",
    "co2_total_kg",
]].drop_duplicates(subset=["route_id"]).copy()

routes_ddl = """\
-- routes table (Postgres)
CREATE TABLE IF NOT EXISTS routes (
  route_id               UUID PRIMARY KEY,
  airline_id             TEXT NOT NULL,
  source_airport_id      BIGINT NOT NULL,
  destination_airport_id BIGINT NOT NULL,
  stops                  INTEGER NOT NULL DEFAULT 0,
  plane_iso              TEXT,

  CONSTRAINT routes_airline_fk  FOREIGN KEY (airline_id) REFERENCES airlines(airline_id),
  CONSTRAINT routes_source_fk   FOREIGN KEY (source_airport_id) REFERENCES airports(airport_id),
  CONSTRAINT routes_dest_fk     FOREIGN KEY (destination_airport_id) REFERENCES airports(airport_id),
  CONSTRAINT routes_plane_fk    FOREIGN KEY (plane_iso) REFERENCES aircraft_types(plane_iso)
);

CREATE INDEX IF NOT EXISTS routes_src_idx  ON routes (source_airport_id);
CREATE INDEX IF NOT EXISTS routes_dst_idx  ON routes (destination_airport_id);
CREATE INDEX IF NOT EXISTS routes_airline_idx ON routes (airline_id);

-- optional uniqueness on the natural key
CREATE UNIQUE INDEX IF NOT EXISTS routes_natural_uq
  ON routes (airline_id, source_airport_id, destination_airport_id, stops, plane_iso);
"""

route_metrics_ddl = """\
-- route_metrics table (Postgres)
CREATE TABLE IF NOT EXISTS route_metrics (
  route_id        UUID PRIMARY KEY REFERENCES routes(route_id) ON DELETE CASCADE,
  distance_km     DOUBLE PRECISION,
  distance_miles  DOUBLE PRECISION,
  is_international BOOLEAN,
  co2_total_kg    DOUBLE PRECISION
);
"""

routes_rows = []
for r in routes_df.itertuples(index=False):
    routes_rows.append(
        "INSERT INTO routes (route_id, airline_id, source_airport_id, destination_airport_id, stops, plane_iso)\n"
        f"VALUES ({sql_quote(r.route_id)}, {sql_quote(r.airline_id)}, {to_num(r.route_source_airport_id)}, "
        f"{to_num(r.route_destination_airport_id)}, {to_num(r.route_stops)}, {sql_quote(r.route_plane_iso)});\n"
    )
routes_inserts = "-- routes seed data\n" + "".join(routes_rows)

metrics_rows = []
for r in route_metrics_df.itertuples(index=False):
    metrics_rows.append(
        "INSERT INTO route_metrics (route_id, distance_km, distance_miles, is_international, co2_total_kg)\n"
        f"VALUES ({sql_quote(r.route_id)}, {to_num(r.distance_km)}, {to_num(r.distance_miles)}, "
        f"{to_bool(r.is_international)}, {to_num(r.co2_total_kg)});\n"
    )
metrics_inserts = "-- route_metrics seed data\n" + "".join(metrics_rows)

with open(os.path.join(SQL_DIR, "007_create_routes.sql"), "w", encoding="utf-8") as f:
    f.write(routes_ddl)

with open(os.path.join(SQL_DIR, "008_insert_routes.sql"), "w", encoding="utf-8") as f:
    f.write(routes_inserts)

with open(os.path.join(SQL_DIR, "009_create_route_metrics.sql"), "w", encoding="utf-8") as f:
    f.write(route_metrics_ddl)

with open(os.path.join(SQL_DIR, "010_insert_route_metrics.sql"), "w", encoding="utf-8") as f:
    f.write(metrics_inserts)

print(f"Wrote routes + route_metrics SQL to: {os.path.abspath(SQL_DIR)}")
print(f"Routes rows: {len(routes_df):,} | Route metrics rows: {len(route_metrics_df):,}")


Wrote aircraft_types SQL | rows: 29
Wrote routes + route_metrics SQL to: /Users/darius/Documents/projects/portfolio/flighter/data-mining/sql_statements
Routes rows: 20,471 | Route metrics rows: 20,471


# Synthetic `flight_schedules` Generation Algorithm

*(Demand + Reputation + Route Realism)*

This document describes an **end-to-end synthetic schedule generator** that produces **Postgres `INSERT` SQL** for a `flight_schedules` table, using:

* **Your main routes dataset** (`df`) with route / airline / airport / aircraft / metrics columns
* A **city-pair frequency CSV** (e.g. `city_pairs.csv`) with columns:
  `city_a, city_b, count`

The generator is designed to be **explainable**, **tunable**, and **realism-oriented**.

---

## 0) Concept: What a “flight schedule” represents

A `route` in your database means:

> “This airline can fly A → B (possibly with stops) using aircraft type X.”

A booking or search system needs something closer to reality: **repeating services** that operate weekly with departure times.

A `flight_schedules` row represents a **repeating operating pattern**, for example:

* Airline: BA
* Route: JFK → LHR
* Flight number: BA117
* Departs: 19:35
* Arrives: 07:25
* Operates: Mon Tue Wed Thu Fri Sat Sun
* Weekly frequency: 7

If a route operates multiple times per day, we generate **multiple schedule rows**, not a single aggregated row.

---

## 1) Inputs and Pre-processing

### 1.1 Market Definition: City-Pairs

Each city-pair `(A, B)` is treated as a **market**.

To avoid double counting `(A, B)` vs `(B, A)`, we canonicalize markets as:

```
market_key = tuple(sorted([city_a, city_b]))
```

---

### 1.2 Aggregating Routes into Markets

From `df`, we build the market key using:

* `source_port_city`
* `destination_port_city`

This allows us to connect **city-pair demand** to the **set of available routes** that can serve that market.

---

## 2) Converting City-Pair Demand into Weekly Flights

The `count` column in your city-pair CSV is treated as a **demand proxy**:

* Higher count ⇒ busier market ⇒ more total departures per week

We convert demand into total weekly flights using:

* **Log normalization** to compress extreme values
* A **calibration curve** mapping normalized demand into a frequency band

The result is:

```
market_weekly_flights
```

This represents the **total number of departures per week across all airlines** in that market.

This step is intentionally tunable so you can make the simulated network feel:

* Denser (hub-heavy, competitive)
* Sparser (regional, low-frequency)

---

## 3) Airline Reputation / Strength Score

To allocate more schedules to stronger airlines, we infer a **reputation proxy** directly from `df`.

For each airline we compute:

1. **Network size**
   Number of routes operated
   → Larger networks imply larger airlines

2. **International share**
   Mean of `is_international`
   → International operations correlate with stronger carriers

3. **Fleet capability proxy**
   Share of high-capacity aircraft (`capacity_max`)
   → Larger aircraft imply higher market strength

4. **Efficiency / modernity proxy**
   Lower `co2_g_per_pax_mile`
   → Newer fleets tend to be more efficient

Each metric is normalized to `[0, 1]` and combined into a single score.

This score is mapped into a multiplicative factor:

```
reputation_factor ∈ [0.6 … 1.3]
```

The factor biases frequency allocation toward stronger airlines in competitive markets.

---

## 4) Route Realism Weights

Within a market, multiple routes may compete. Each candidate route receives a **route weight**:

```
route_weight =
    reputation_factor(airline)
  × hub_bonus
  × capacity_gauge_factor
  × stops_penalty
  × international_bias
```

Where:

### Hub Bonus

Airlines fly more frequently from their hubs.

Hubs are inferred from the share of an airline’s routes originating in each city.

---

### Capacity Gauge Factor

Higher-capacity aircraft slightly reduce the need for frequency but also signal strength.

We apply a **square-root scaling** to keep effects soft and realistic.

---

### Stops Penalty

Multi-stop routes receive lower frequency allocations.

---

### International Bias

International routes typically operate less frequently than dense domestic shuttles.

---

All of these adjustments are **gentle modifiers** — demand and airline reputation remain the dominant drivers.

---

## 5) Allocating Market Flights to Routes

For each market:

1. Compute `market_weekly_flights` from demand
2. Identify all candidate routes in `df` that serve the city-pair (either direction)
3. Compute `route_weight` for each candidate
4. Convert weights into shares and allocate integer frequencies using **largest-remainder rounding**:

   * Assign `floor(share × market_weekly_flights)`
   * Distribute remaining flights to routes with the largest fractional remainders

**Result:**
Each `(airline, route)` receives an integer `allocated_weekly_flights`.

---

## 6) Expanding Weekly Flights into Schedule Rows

A route may be allocated, for example, 20 flights per week.

Instead of one row with `20`, we generate **multiple schedule rows**:

1. Allocate as many **daily (7/wk)** schedules as possible
2. Allocate remaining flights as **partial-week schedules** (e.g. 3/wk, 5/wk)

For each schedule row we generate:

### Operating Days

A 7-bit mask (Mon–Sun):

* Daily: `127` (`1111111`)
* Weekdays: `31` (`0011111`)
* Remainders: random valid subsets

---

### Departure Time

Based on distance:

* Short-haul: morning and evening peaks
* Long-haul: midday and red-eye bias

---

### Arrival Time

Estimated using a simple block-time model:

```
duration_hours ≈ distance_km / cruise_speed + overhead
arrival_time = depart_time + duration
```

Time zones are **intentionally ignored** for now to keep the model simple.

---

### Flight Number

Deterministically derived from:

* Airline code
* Route hash

Ensures stability across regeneration runs.

---

## 7) Output: SQL Insert File

The generator writes **Postgres INSERT statements only** (no DB writes from Python) to:

```
./sql_statements/011_generate_flight_schedules.sql
```

---

## 8) Explicit Assumptions & Limitations

* Schedules are **synthetic** and plausible, not real-world replicas
* Arrival times ignore timezone differences
* City-pairs not present in `df` are skipped
* Routes with no demand entry receive a small default demand

---

## 9) How to Tune the Generator

Key parameters you can adjust in code:

* `MIN_MARKET_WEEKLY`, `MAX_MARKET_WEEKLY` — overall network density
* Demand curve parameters (`DEMAND_ALPHA`, `DEMAND_BETA`)
* Reputation weights (`REP_W_*`)
* Penalties (stops, international bias)
* Schedule splitting rules (daily vs partial-week patterns)


In [7]:
# ----------------------------
# Configuration
# ----------------------------
SQL_DIR = "./sql_statements"
os.makedirs(SQL_DIR, exist_ok=True)

SCRIPT_NAME = "generate_flight_schedules"
OUT_SQL_PATH = os.path.join(SQL_DIR, f"011_{SCRIPT_NAME}.sql")

# Path to your city-pair demand CSV:
# Must have columns: city_a, city_b, count
CITYPAIR_CSV_PATH = "./clean_data/busiest_city_pairs.csv"

# Effective date window for generated schedules (synthetic, tunable)
EFFECTIVE_FROM = date(2026, 1, 1).isoformat()
EFFECTIVE_TO   = date(2026, 12, 31).isoformat()

# Demand → weekly flights tuning
MIN_MARKET_WEEKLY = 2     # smallest markets: 2 flights/week
MAX_MARKET_WEEKLY = 140   # biggest markets: up to 20/day across all airlines (synthetic upper bound)

# Curve parameters:
# demand_norm in [0,1]
# market_weekly = MIN + (MAX-MIN) * (demand_norm ** DEMAND_ALPHA) * (0.5 + 0.5*demand_norm) ** DEMAND_BETA
DEMAND_ALPHA = 0.55
DEMAND_BETA  = 0.65

# Airline reputation weights (sum roughly 1.0)
REP_W_NETWORK = 0.50
REP_W_INTL    = 0.20
REP_W_CAP     = 0.20
REP_W_ECO     = 0.10

# Route weight modifiers
STOPS_PENALTY_PER_STOP = 0.35     # each stop reduces attractiveness
INTL_FREQ_MULTIPLIER   = 0.75     # international routes tend to be less frequent
HUB_BONUS_MULTIPLIER   = 0.30     # hub effect strength
GAUGE_SOFTENING        = 0.35     # how strongly capacity influences weight (very gentle)

# Schedule generation
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Cruise speed assumptions for time estimation (km/h)
CRUISE_SPEED_SHORT = 720
CRUISE_SPEED_MED   = 800
CRUISE_SPEED_LONG  = 860
OVERHEAD_HOURS     = 0.6  # taxi/climb/descend etc.

# ----------------------------
# Helpers
# ----------------------------
def sql_quote(v):
    """SQL literal quoting for text/date/time."""
    if v is None or (isinstance(v, float) and pd.isna(v)) or (pd.isna(v) if not isinstance(v, str) else False):
        return "NULL"
    s = str(v).replace("'", "''")
    return f"'{s}'"

def to_num(v):
    if v is None or (isinstance(v, float) and pd.isna(v)) or (pd.isna(v) if not isinstance(v, str) else False):
        return "NULL"
    if isinstance(v, (np.integer, int)):
        return str(int(v))
    if isinstance(v, (np.floating, float)):
        # avoid scientific notation in SQL
        return format(float(v), "f").rstrip("0").rstrip(".") if "." in format(float(v), "f") else str(float(v))
    return str(v)

def to_bool(v):
    if v is None or (isinstance(v, float) and pd.isna(v)) or (pd.isna(v) if not isinstance(v, str) else False):
        return "NULL"
    return "TRUE" if bool(v) else "FALSE"

def normalize_series(s: pd.Series) -> pd.Series:
    """Min-max normalize to [0,1], safe for constant series."""
    s = s.astype(float)
    mn, mx = float(s.min()), float(s.max())
    if math.isclose(mn, mx):
        return pd.Series([0.5] * len(s), index=s.index)
    return (s - mn) / (mx - mn)

def stable_uuid_from_key(key: str) -> str:
    """Deterministic UUID from a string key."""
    return str(uuid.uuid5(uuid.NAMESPACE_URL, key))

def stable_int_from_key(key: str) -> int:
    """Deterministic int from a string key (for flight numbers)."""
    h = hashlib.sha1(key.encode("utf-8")).digest()
    return int.from_bytes(h[:8], "big", signed=False)

def stable_route_id(airline_id, src_airport_id, dst_airport_id, stops, plane_iso) -> str:
    """Deterministic route_id derived from the natural keys (UUID)."""
    key = f"ROUTE|{airline_id}|{src_airport_id}|{dst_airport_id}|{stops}|{plane_iso}"
    return stable_uuid_from_key(key)

def canonical_citypair(a: str, b: str) -> tuple:
    """Canonical market key for a pair of cities (unordered)."""
    if a is None or b is None or pd.isna(a) or pd.isna(b):
        return (None, None)
    a = str(a).strip()
    b = str(b).strip()
    return tuple(sorted([a, b]))

def demand_to_market_weekly(demand_norm: float) -> int:
    """Map normalized demand [0,1] to a plausible market weekly flight total."""
    demand_norm = float(np.clip(demand_norm, 0.0, 1.0))
    val = MIN_MARKET_WEEKLY + (MAX_MARKET_WEEKLY - MIN_MARKET_WEEKLY) * \
          (demand_norm ** DEMAND_ALPHA) * ((0.5 + 0.5 * demand_norm) ** DEMAND_BETA)
    return int(round(val))

def pick_operating_days_mask(k: int) -> int:
    """
    Choose k operating days out of 7 as a bitmask (Mon..Sun => bits 0..6).
    Example: daily => 127 (0b1111111)
    """
    k = int(max(1, min(7, k)))
    days = list(range(7))  # 0=Mon ... 6=Sun
    random.shuffle(days)
    chosen = sorted(days[:k])
    mask = 0
    for d in chosen:
        mask |= (1 << d)
    return mask

def time_to_str(minutes_since_midnight: int) -> str:
    minutes_since_midnight %= (24 * 60)
    hh = minutes_since_midnight // 60
    mm = minutes_since_midnight % 60
    return f"{hh:02d}:{mm:02d}"

def estimate_block_time_minutes(distance_km: float) -> int:
    """Estimate block time (minutes) from distance_km with crude cruise speed bands."""
    if distance_km is None or pd.isna(distance_km) or distance_km <= 0:
        return int(round((1.5 + OVERHEAD_HOURS) * 60))
    if distance_km < 800:
        speed = CRUISE_SPEED_SHORT
    elif distance_km < 3500:
        speed = CRUISE_SPEED_MED
    else:
        speed = CRUISE_SPEED_LONG
    hours = (float(distance_km) / speed) + OVERHEAD_HOURS
    return int(round(hours * 60))

def choose_departure_minutes(distance_km: float, is_international: bool) -> int:
    """
    Choose a plausible departure time (minutes since midnight) based on distance and intl.
    - short haul: peaks around 06-09 and 16-20
    - medium haul: spread across day
    - long haul: midday and evening/red-eye bias
    """
    r = random.random()
    if distance_km is None or pd.isna(distance_km):
        distance_km = 1000

    if distance_km < 800 and not is_international:
        # commuter peaks
        if r < 0.55:
            # morning peak
            return random.randint(6 * 60, 9 * 60 + 30)
        else:
            # evening peak
            return random.randint(16 * 60, 20 * 60 + 30)
    elif distance_km < 3500:
        # spread
        if r < 0.20:
            return random.randint(6 * 60, 9 * 60 + 30)
        elif r < 0.70:
            return random.randint(10 * 60, 16 * 60 + 30)
        else:
            return random.randint(17 * 60, 22 * 60)
    else:
        # long haul: midday + evening/red-eye
        if r < 0.45:
            return random.randint(11 * 60, 15 * 60 + 30)
        else:
            return random.randint(18 * 60, 23 * 60 + 30)

def make_flight_number(airline_iata: str, airline_icao: str, key: str) -> str:
    """
    Deterministic-ish flight number:
    prefer IATA (2 chars), else ICAO (3 chars), else 'XX'
    number: 3-4 digits derived from hash
    """
    prefix = None
    if isinstance(airline_iata, str) and airline_iata.strip():
        prefix = airline_iata.strip().upper()
    elif isinstance(airline_icao, str) and airline_icao.strip():
        prefix = airline_icao.strip().upper()
    else:
        prefix = "XX"

    n = stable_int_from_key(key) % 9000 + 100  # 100..9099
    return f"{prefix}{n}"

def largest_remainder_allocation(total: int, weights: np.ndarray) -> np.ndarray:
    """
    Allocate integer counts that sum to total, proportional to weights.
    Uses largest remainder method.
    """
    total = int(max(0, total))
    if total == 0 or weights.sum() <= 0:
        return np.zeros_like(weights, dtype=int)

    shares = weights / weights.sum()
    raw = shares * total
    base = np.floor(raw).astype(int)
    remainder = raw - base

    deficit = total - base.sum()
    if deficit > 0:
        idx = np.argsort(-remainder)  # descending remainder
        for i in idx[:deficit]:
            base[i] += 1
    return base

# ----------------------------
# 1) Load and normalize demand table
# ----------------------------
citypairs = pd.read_csv(CITYPAIR_CSV_PATH)
required_cols = {"city_a", "city_b", "count"}
missing = required_cols - set(citypairs.columns)
if missing:
    raise ValueError(f"City-pair CSV missing columns: {missing}")

citypairs["city_a"] = citypairs["city_a"].astype("string").str.strip()
citypairs["city_b"] = citypairs["city_b"].astype("string").str.strip()
citypairs["market_key"] = citypairs.apply(lambda r: canonical_citypair(r["city_a"], r["city_b"]), axis=1)

# Combine duplicates safely
demand_by_market = (
    citypairs.groupby("market_key", as_index=True)["count"]
    .sum()
    .sort_values(ascending=False)
)

# Normalize demand with log scaling to reduce extreme skew
log_demand = np.log1p(demand_by_market.astype(float))
demand_norm = normalize_series(log_demand)

market_weekly = demand_norm.apply(demand_to_market_weekly).astype(int)

# ----------------------------
# 2) Build route candidates with market keys from df
# ----------------------------
routes = df.copy()

# Canonical market key by cities
routes["market_key"] = routes.apply(
    lambda r: canonical_citypair(r.get("source_port_city"), r.get("destination_port_city")),
    axis=1
)

# Deterministic route_id (so schedules can reference it)
routes["route_id"] = routes.apply(
    lambda r: stable_route_id(
        r.get("airline_id"),
        r.get("route_source_airport_id"),
        r.get("route_destination_airport_id"),
        r.get("route_stops"),
        r.get("route_plane_iso"),
    ),
    axis=1
)

# If a market appears in routes but not in demand CSV, give it a small default demand
# (this prevents "dead markets" if your CSV doesn't cover everything)
all_markets_in_routes = set(routes["market_key"].dropna().unique().tolist())
missing_markets = all_markets_in_routes - set(market_weekly.index.tolist())
if missing_markets:
    # default demand is low: around 2-6 flights/week depending on distance
    # we implement default as a small constant market_weekly = MIN_MARKET_WEEKLY
    # (you can tune this)
    for mk in missing_markets:
        market_weekly.loc[mk] = MIN_MARKET_WEEKLY

# ----------------------------
# 3) Compute airline reputation factor from df (proxy)
# ----------------------------
# Network size
routes_per_airline = routes.groupby("airline_id").size()

# International share
intl_share = routes.groupby("airline_id")["is_international"].mean().fillna(0.0)

# Fleet capability proxy: share of routes with capacity_max > 250 (widebody-ish)
is_wide = (routes["capacity_max"].fillna(0) > 250).astype(int)
wide_share = routes.assign(is_wide=is_wide).groupby("airline_id")["is_wide"].mean().fillna(0.0)

# Eco proxy: lower CO2 per pax-mile is better
eco_raw = routes.groupby("airline_id")["co2_g_per_pax_mile"].mean()
eco_norm = 1.0 - normalize_series(eco_raw.fillna(eco_raw.median() if not eco_raw.dropna().empty else 0.0))

rep = (
    REP_W_NETWORK * normalize_series(routes_per_airline) +
    REP_W_INTL    * normalize_series(intl_share) +
    REP_W_CAP     * normalize_series(wide_share) +
    REP_W_ECO     * eco_norm
).clip(0.0, 1.0)

# map to multiplicative factor
reputation_factor = (0.6 + 0.7 * rep).to_dict()  # 0.6..1.3

# ----------------------------
# 4) Infer hubs: hub strength per (airline, source_city)
# ----------------------------
airline_city_origin = routes.groupby(["airline_id", "source_port_city"]).size()
airline_total = routes.groupby("airline_id").size()

hub_strength = (airline_city_origin / airline_city_origin.index.get_level_values(0).map(airline_total)).fillna(0.0)
# hub_strength is in [0,1], higher means "this city is a hub for this airline"

def get_hub_bonus(airline_id, source_city) -> float:
    try:
        hs = float(hub_strength.loc[(airline_id, source_city)])
    except Exception:
        hs = 0.0
    return 1.0 + HUB_BONUS_MULTIPLIER * hs  # mild boost

# ----------------------------
# 5) Allocate market weekly flights → route-level frequencies
# ----------------------------
schedule_rows = []

# Pre-group routes by market for speed
routes_by_market = {mk: g for mk, g in routes.groupby("market_key")}

for mk, total_weekly in market_weekly.items():
    if mk not in routes_by_market:
        continue

    cand = routes_by_market[mk].copy()
    if cand.empty:
        continue

    # Compute route weights
    # reputation: per airline
    cand["rep_factor"] = cand["airline_id"].map(lambda x: reputation_factor.get(x, 1.0))

    # hub bonus uses source city
    cand["hub_bonus"] = cand.apply(lambda r: get_hub_bonus(r["airline_id"], r["source_port_city"]), axis=1)

    # stops penalty
    stops = cand["route_stops"].fillna(0).astype(int)
    cand["stops_factor"] = (1.0 / (1.0 + STOPS_PENALTY_PER_STOP * stops)).clip(0.1, 1.0)

    # international frequency multiplier (applied as a mild weight reduction)
    cand["intl_factor"] = cand["is_international"].fillna(False).astype(bool).map(
        lambda x: INTL_FREQ_MULTIPLIER if x else 1.0
    )

    # gauge factor: we want this gentle; higher capacity slightly increases "ability" but not linearly
    cap = cand["capacity_max"].fillna(cand["capacity_max"].median() if not cand["capacity_max"].dropna().empty else 150)
    cap = cap.clip(lower=50)
    cand["gauge_factor"] = (cap ** GAUGE_SOFTENING) / float((cap ** GAUGE_SOFTENING).median())

    cand["route_weight"] = (
        cand["rep_factor"] *
        cand["hub_bonus"] *
        cand["stops_factor"] *
        cand["intl_factor"] *
        cand["gauge_factor"]
    ).astype(float)

    weights = cand["route_weight"].to_numpy(dtype=float)
    alloc = largest_remainder_allocation(int(total_weekly), weights)

    cand = cand.assign(allocated_weekly=alloc)
    cand = cand[cand["allocated_weekly"] > 0]

    # ----------------------------
    # 6) Expand allocations into multiple schedule rows
    # ----------------------------
    for r in cand.itertuples(index=False):
        remaining = int(r.allocated_weekly)

        # decide how many daily schedules (7/wk) to create
        daily_count = remaining // 7
        remainder = remaining % 7

        # if very large, allow multiple daily schedules (e.g., 21/wk => 3 daily)
        schedule_chunks = [7] * daily_count
        if remainder > 0:
            schedule_chunks.append(remainder)

        # If the market is very small (1-2/week), keep it as one partial schedule
        # already covered by chunks

        for chunk_idx, wf in enumerate(schedule_chunks):
            # operating days mask
            if wf >= 7:
                op_mask = 127  # 1111111
            else:
                # for partial, choose wf days
                op_mask = pick_operating_days_mask(wf)

            # derive times
            dist_km = r.distance_km
            intl = bool(r.is_international) if not pd.isna(r.is_international) else False

            dep_min = choose_departure_minutes(dist_km, intl)
            block_min = estimate_block_time_minutes(dist_km)
            arr_min = dep_min + block_min  # timezone differences ignored (explicit)

            dep_time = time_to_str(dep_min)
            arr_time = time_to_str(arr_min)

            # flight number (stable-ish)
            fn_key = f"{r.airline_id}|{r.route_id}|{chunk_idx}|{dep_time}|{op_mask}"
            flight_number = make_flight_number(r.airline_iata, r.airline_icao, fn_key)

            # schedule_id (deterministic UUID)
            schedule_id = stable_uuid_from_key(f"SCHED|{fn_key}")

            schedule_rows.append({
                "flight_schedule_id": schedule_id,
                "route_id": r.route_id,
                "airline_id": r.airline_id,
                "flight_number": flight_number,
                "depart_time_local": dep_time,
                "arrive_time_local": arr_time,
                "operating_days_mask": int(op_mask),
                "weekly_frequency": int(wf),
                "aircraft_iso": r.route_plane_iso,
                "effective_from": EFFECTIVE_FROM,
                "effective_to": EFFECTIVE_TO,
            })

# Convert to DataFrame and de-duplicate just in case
schedules_df = pd.DataFrame(schedule_rows).drop_duplicates(subset=["flight_schedule_id"]).copy()

# ----------------------------
# 7) Generate SQL INSERT file
# ----------------------------
flight_schedules_ddl = """
CREATE TABLE IF NOT EXISTS flight_schedules (
    flight_schedule_id UUID PRIMARY KEY,

    route_id   UUID NOT NULL,
    airline_id TEXT NOT NULL,

    flight_number VARCHAR(8) NOT NULL,

    depart_time_local TIME NOT NULL,
    arrive_time_local TIME NOT NULL,

    operating_days_mask SMALLINT NOT NULL
        CHECK (operating_days_mask BETWEEN 1 AND 127),

    weekly_frequency SMALLINT NOT NULL
        CHECK (weekly_frequency > 0 AND weekly_frequency <= 21),

    aircraft_iso CHAR(3) NOT NULL,

    effective_from DATE NOT NULL,
    effective_to   DATE NOT NULL,

    CONSTRAINT chk_effective_dates
        CHECK (effective_to >= effective_from)
);
"""


sql_lines = []
sql_lines.append(f"-- Synthetic flight schedules generated from demand + reputation + route realism")
sql_lines.append(f"-- Source demand file: {CITYPAIR_CSV_PATH}")
sql_lines.append(f"-- Output date range: {EFFECTIVE_FROM} to {EFFECTIVE_TO}")
sql_lines.append(f"-- Rows: {len(schedules_df):,}")
sql_lines.append("")

sql_lines.append(flight_schedules_ddl + "\n\n")

for r in schedules_df.itertuples(index=False):
    sql_lines.append(
        "INSERT INTO flight_schedules "
        "(flight_schedule_id, route_id, airline_id, flight_number, depart_time_local, arrive_time_local, "
        "operating_days_mask, weekly_frequency, aircraft_iso, effective_from, effective_to)\n"
        f"VALUES ({sql_quote(r.flight_schedule_id)}, {sql_quote(r.route_id)}, {sql_quote(r.airline_id)}, "
        f"{sql_quote(r.flight_number)}, {sql_quote(r.depart_time_local)}, {sql_quote(r.arrive_time_local)}, "
        f"{to_num(r.operating_days_mask)}, {to_num(r.weekly_frequency)}, {sql_quote(r.aircraft_iso)}, "
        f"{sql_quote(r.effective_from)}, {sql_quote(r.effective_to)});\n"
    )

with open(OUT_SQL_PATH, "w", encoding="utf-8") as f:
    f.write("\n".join(sql_lines))

print(f"✅ Wrote flight_schedules INSERT SQL: {os.path.abspath(OUT_SQL_PATH)}")
print(f"Generated schedules: {len(schedules_df):,}")
print("Sample rows:")
display(schedules_df.head(10))


✅ Wrote flight_schedules INSERT SQL: /Users/darius/Documents/projects/portfolio/flighter/data-mining/sql_statements/011_generate_flight_schedules.sql
Generated schedules: 56,919
Sample rows:


,flight_schedule_id,route_id,airline_id,flight_number,depart_time_local,arrive_time_local,operating_days_mask,weekly_frequency,aircraft_iso,effective_from,effective_to
0,d12306b6-dde3-599d-b21e-1e27b3fb4de1,2815ac1f-e72d-52be-af25-841356236ef7,3674,DD2278,06:26,07:58,95,6,738,2026-01-01,2026-12-31
1,18e9b1bb-28ca-52ce-a843-4318440c38b6,df01f2a1-57f0-5b43-9897-c1f2170e75d4,3674,DD5529,16:44,18:16,127,7,738,2026-01-01,2026-12-31
2,7b985621-d2b7-5a18-bafc-b40798214e9a,cd263f06-d50c-547d-af99-ddc1d5a4e162,4947,FD3445,16:16,17:48,127,7,320,2026-01-01,2026-12-31
3,43cb51e9-4630-5b66-b009-80ba72ebea05,781a81ac-a804-5ff5-9501-317a87226adb,4947,FD8110,06:55,08:27,127,7,320,2026-01-01,2026-12-31
4,6999b418-6f83-5bb1-b272-d551f6ee15c6,43b07bce-78ce-59d2-bd36-1ee98fa441c0,2987,JL8117,08:34,10:09,127,7,734,2026-01-01,2026-12-31
5,5b6caca3-13e4-559f-9b80-235981fabdfc,919b3b4d-500f-590c-b7a8-fe13fe3754d8,2987,JL4663,06:50,08:25,127,7,734,2026-01-01,2026-12-31
6,4d59f162-3d6d-586a-925a-0bd751bb2b04,622fd061-37c7-5d98-bf83-9efa70dab28f,3090,KL5802,16:03,17:38,95,6,320,2026-01-01,2026-12-31
7,3824ba8b-7d0e-5faa-9bbc-f8573ff1515a,38078b64-5db3-5add-a832-7b26e455ce18,3090,KL5049,09:15,10:50,63,6,320,2026-01-01,2026-12-31
8,176490ec-5f6c-57dd-80a3-654a26a10338,86e8746d-4909-5eeb-b608-5d2f5ebcf8af,3378,MH4857,06:23,07:58,127,7,320,2026-01-01,2026-12-31
9,b8b4a37b-385a-56b4-a587-e4a070a68c09,bc41150c-dcc5-5395-9940-edfdb026f219,3378,MH6013,07:31,09:06,127,7,320,2026-01-01,2026-12-31
